In [2]:
import json
import csv
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keplergl

In [3]:
def read_partition(partition_fname):
    """
    Reads a partition (district map) from a csv file with 
    columns 'GEOID' (unique identifiers for the census tracts) and 'district'. 
    
    Returns a pandas DataFrame object representing the map, where
    'GEOID' is the key.
    """
    with open(partition_fname, 'r') as map_file:
        map_reader = csv.reader(map_file)
        map_raw = list(map_reader)

    map_headers = map_raw.pop(0)
    map_df = pd.DataFrame(map_raw, columns=map_headers).astype({'district': int})
    return map_df.set_index('GEOID')

In [4]:
def create_and_get_gdf(tracts_fname, partition_fname):
    """
    Loads a sample Wisconsin district map. 

    Parameters:
        tracts_fname: the name of the Wisconsin census tracts file (zipped shapefile)
        partition_fname: the name of the initial partition file (.csv)

    Returns:
        the given Wisconsin district map as a GeoPandas DataFrame object
    """
    # Step 1. Build a GeoDataFrame (a geographic version of a pandas DataFrame)
    # from the tracts zip file (adding 'zip://' prefix if missing). 
    tracts_fname = tracts_fname if 'zip://' in tracts_fname else 'zip://' + tracts_fname
    gdf = geopandas.read_file(tracts_fname)
    gdf.set_index('GEOID', inplace=True)

    # Step 2. Load the initial partition and join it to the GeoDataFrame. 
    map_df = read_partition(partition_fname)
    map_gdf = gdf.join(map_df)

    map_gdf['district'].fillna(value=-1, inplace=True) # Slight cleanup
    
    return map_gdf

In [5]:
def remove_unnecessary_gdf_columns(map_as_gdf):
    map_as_gdf.pop('NAME')
    map_as_gdf.pop('STATEFP')
    map_as_gdf.pop('COUNTYFP')
    map_as_gdf.pop('MTFCC')
    map_as_gdf.pop('FUNCSTAT')
    map_as_gdf.pop('TRACTCE')
    map_as_gdf.pop('INTPTLAT')
    map_as_gdf.pop('INTPTLON')
    map_as_gdf.pop('ALAND')
    map_as_gdf.pop('AWATER')
    map_as_gdf.pop('district')
    
    return map_as_gdf

In [6]:
def read_json(json_fname):
    # read and load in json data into python dictionary
    with open(json_fname, 'r') as json_data:
        all_json_data=json_data.read()
        
    all_json_dict = json.loads(all_json_data)
    
    return all_json_dict

In [7]:
def get_list_of_flip_dict(json_fname):
    
    """
    Gets a list of python dictionaries each containing the information for one flip.
    
    Parameters: json_fname: the name of json file containing flip information

    Returns: list of of python dictionaries each containing the information for one flip
    """
    all_json_dict = read_json(json_fname)    
    
    # gets list of all single flip dictionaries stored in json
    flip_json_list_of_dict = all_json_dict['flips']
    
    return flip_json_list_of_dict

In [8]:
def get_initial_map_data(json_fname):
    
    """
    Gets a python dictionary containing district numbers as keys mapped to GEOIDs in a list.
    
    Parameters: json_fname: the name of json file containing map data information

    Returns: python dictionary containing district numbers as keys mapped to GEOIDs in a list
    """
    
    all_json_dict = read_json(json_fname)    
    
    # get list of single flip dictionaries stored in json
    map_data_dict = all_json_dict['initial_map']
    
    return map_data_dict

In [9]:
def add_initial_map_column_to_gdf(map_data_dict, map_as_gdf):      
    """
    Adds a column of district data read in from the initial map key to GeoPandas DataFrame object map passed in
    and another column noting the district plan number, in this case, 1
    
    Parameters: 
            map_data_dict: a list that contains a dictionary of initial map data with district numbers as keys and 
            GEOIDs as values
            map_as_gdf: GeoPandas DataFrame object that encodes data for a map
    
    Returns: original GeoPandas DataFrame with the new initial map data added
    """
        
    column_name = 'district' # previously district_plan_0
    list_of_districts = list(map_data_dict.keys()) # gets list of districts from dictionary district and GEOID data
    # creates a list of int of -1 of the length of the number of rows in map_as_gdf
    empty_list_for_column = [-1] * len(map_as_gdf.index)
    map_as_gdf.insert(len(map_as_gdf.columns), column_name, empty_list_for_column)
    
    # sets district to geoid in new column space based on json data
    for district_number in list_of_districts:
        list_of_geoids = map_data_dict[district_number] # gets list of GEOIDs in a given district
        
        for geoid in list_of_geoids:
            map_as_gdf.loc[geoid, column_name] = float(district_number)
    
    # adds column with district plan number to gdf –– 1 in this case
    plan_name = 'plan'
    plan_number_list_for_column = [1] * len(map_as_gdf.index)

    map_as_gdf.insert(len(map_as_gdf.columns), plan_name, plan_number_list_for_column)
    
    return map_as_gdf

In [1]:
def implement_flip_into_original_gdf(one_flip_dictionary, map_as_gdf):
    
    """
    Alters a GeoPandas Dataframe object map to reflect information in a flip dictionary
    
    Parameters: 
            list_of_one_flip_dict: list that contains one dictionary describing one flip
            map_as_gdf: GeoPandas DataFrame object that encodes data for a map
    
    Returns: new GeoPandas DataFrame with the flip information implemented into data
    """  
    
    list_of_single_geoid = list(one_flip_dictionary.keys())
    geoid_key = list_of_single_geoid[0] # gets value of GEOID in the flip dictionary
    new_district_value = one_flip_dictionary[geoid_key] # gets value of new district as float from flip dictionary
    
    map_as_gdf.loc[geoid_key, 'district'] = float(new_district_value)
    
    return map_as_gdf

In [10]:
def implement_flip_into_new_gdf(one_flip_dictionary, map_as_gdf, district_counter):
    
    """
    Duplicates initial district column in a GeoPandas Dataframe object map and creates a copy
    and alters and returns the copy to reflect information in a flip dictionary
    
    Parameters: 
            list_of_one_flip_dict: list that contains one dictionary describing one flip
            map_as_gdf: GeoPandas DataFrame object that encodes data for a map
            district_counter: int of district plan
    
    Returns: new GeoPandas DataFrame with the flip information implemented into data
    """
    # duplicates initial district column and names it whatever column_name stores
    new_map_as_gdf = map_as_gdf.copy()
    
    list_of_single_geoid = list(one_flip_dictionary.keys())
    geoid_key = list_of_single_geoid[0] # gets value of GEOID in the flip dictionary
    new_district_value = one_flip_dictionary[geoid_key] # gets value of new district as float from flip dictionary
    
    new_map_as_gdf.loc[geoid_key, 'district'] = float(new_district_value)
    new_map_as_gdf.loc[:, 'plan'] = district_counter
    
    return new_map_as_gdf